# Plot methods in nlmod

This notebook shows different methods of plotting data with nlmod. 

There are many ways to plot data and it depends on the type of data and plot which of
these method is the most convenient:
- using `nlmod.plot` utilities
- using `flopy` plot methods
- using `xarray` plot methods

The default plot methods in nlmod use a model Dataset as input (this is an xarray
Dataset with some required variables and attributes). These plotting methods are
accessible through `nlmod.plot`.

Flopy contains its own plotting utilities and nlmod contains some wrapper functions that
use flopy's plotting utilities under the hood. These require a flopy modelgrid or model
object. These plotting methods are accessible through `nlmod.plot.flopy`.

Finally, xarray also allows plotting of data with `.plot()`. This is used in a few
cases in this notebook but for more detailed information, refer to the 
[xarray documentation](https://xarray.pydata.org/en/v2023.08.0/gallery.html).

In [ ]:
import os

import flopy
import matplotlib.pyplot as plt
import xarray as xr

import nlmod
from nlmod.plot import DatasetCrossSection

In [ ]:
nlmod.util.get_color_logger("INFO")
nlmod.show_versions()

First we read a fully run model, from the notebook 09_schoonhoven.ipynb. Please run that notebook first.

In [ ]:
model_name = "Schoonhoven"
model_ws = "schoonhoven"
ds = xr.open_dataset(os.path.join(model_ws, f"{model_name}.nc"))

# add calculated heads
ds["head"] = nlmod.gwf.get_heads_da(ds)
ds

For the flopy plot-methods we need a modelgrid object. We generate this from the model Dataset using the method. nlmod.grid.modelgrid_from_ds().

In [ ]:
modelgrid = nlmod.grid.modelgrid_from_ds(ds)
modelgrid

## Maps
We can plot variables on a map using nlmod.plot.data_array(). We can also use the PlotMapView-class from flopy, and plot an array using the plot_array method.

In [ ]:
f, ax = nlmod.plot.get_map(ds.extent, ncols=2)

# plot using nlmod
pc = nlmod.plot.data_array(ds["top"], ds=ds, ax=ax[0])

# plot using flopy
pmv = flopy.plot.PlotMapView(modelgrid=modelgrid, ax=ax[1])
pmv.plot_array(ds["top"])

## Cross-sections
We can also plot cross-sections, either with DatasetCrossSection in nlmod, or using the PlotCrossSection class of flopy.

In [ ]:
y = (ds.extent[2] + ds.extent[3]) / 2 + 0.1
line = [(ds.extent[0], y), (ds.extent[1], y)]
zmin = -100.0
zmax = 10.0

In [ ]:
f, ax = plt.subplots(figsize=(10, 5), nrows=2)

# plot using nlmod
dcs = DatasetCrossSection(ds, line=line, zmin=zmin, zmax=zmax, ax=ax[0])
dcs.plot_array(ds["kh"])

# plot using flopy
pcs = flopy.plot.PlotCrossSection(modelgrid=modelgrid, line={"line": line}, ax=ax[1])
pcs.plot_array(ds["kh"])
pcs.ax.set_ylim((zmin, zmax));

With the DatasetCrossSection in `nlmod` it is also possible to plot the layers according to the official colors of REGIS, to plot the layer names on the plot, or to plot the model grid in the cross-section. An example is shown in the plot below.

The location of the cross-section and the cross-section labels can be added using `nlmod.plot.inset_map()` and `nlmod.plot.add_xsec_line_and_labels()`.

In [ ]:
f, ax = plt.subplots(figsize=(10, 5))
dcs = DatasetCrossSection(ds, line=line, zmin=-200, zmax=10, ax=ax)
colors = nlmod.read.regis.get_legend()
dcs.plot_layers(colors=colors, min_label_area=1000)
dcs.plot_grid(vertical=False, linewidth=0.5)
mapax = nlmod.plot.inset_map(ax, ds.extent)
nlmod.plot.add_xsec_line_and_labels(line, ax, mapax)

## Time series
For time series we use the functionality of xarray, as we have read the heads in a xarray DataArray.

In [ ]:
x = 118228
y = 439870
head_point = nlmod.gwf.get_head_at_point(ds["head"], x=x, y=y, ds=ds)
head_point.plot.line(hue="layer", size=10);

We can also use pandas to plot the heads. First transform the data to a Pandas DataFrame.

In [ ]:
df = head_point.to_pandas()
df

And then plot this DataFrame.

In [ ]:
df.plot(figsize=(10, 10))